In [3]:
import pandas as pd
# Đường dẫn tới file CSV đã được trích xuất từ COCO2017
file_path = "f_coco_triplets.csv"

# Đọc dữ liệu
df = pd.read_csv(file_path)

In [4]:
import networkx as nx

# Tạo đồ thị có hướng
G = nx.DiGraph()

# Thêm node và edge từ DataFrame
for _, row in df.iterrows():
    subj = row['subject']
    obj = row['object']
    rel = row['predicate']
    img_id = row['image_id']

    # Thêm các nút (node)
    G.add_node(subj, type='entity')
    G.add_node(obj, type='entity')
    G.add_node(f"img_{img_id}", type='image')

    # Thêm quan hệ: subject --predicate--> object
    G.add_edge(subj, obj, label=rel)
    
    # Thêm quan hệ: subject và object thuộc về ảnh nào
    G.add_edge(f"img_{img_id}", subj, label='has_object')
    G.add_edge(f"img_{img_id}", obj, label='has_object')

In [5]:
print("Tổng số nút:", G.number_of_nodes())
print("Tổng số cạnh:", G.number_of_edges())

# Ví dụ in 5 cạnh đầu
for u, v, data in list(G.edges(data=True))[:5]:
    print(f"{u} --[{data['label']}]--> {v}")

Tổng số nút: 3458
Tổng số cạnh: 8106
man --[wear]--> hat
man --[enjoy]--> nap
man --[hold]--> banana
man --[take]--> bite
man --[herd]--> goats


In [7]:
from py2neo import Graph, Node, Relationship

# Kết nối đến Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

In [10]:
# Optional: xóa toàn bộ dữ liệu cũ
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [8]:
def create_kg(tx, subj, pred, obj, img_id):
    query = """
    MERGE (s:Entity {name: $subj})
    MERGE (o:Entity {name: $obj})
    MERGE (i:Image {id: $img_id})
    MERGE (i)-[:HAS_OBJECT]->(s)
    MERGE (i)-[:HAS_OBJECT]->(o)
    MERGE (s)-[r:RELATION {name: $pred}]->(o)
    """
    tx.run(query, subj=subj, pred=pred, obj=obj, img_id=str(img_id))

In [11]:
from py2neo import Node, Relationship, NodeMatcher

# Lọc bỏ các dòng chứa NaN trong các cột quan trọng
df_clean = df.dropna(subset=['subject', 'predicate', 'object', 'image_id'])

matcher = NodeMatcher(graph)

for _, row in df_clean.iterrows():
    subj_name = str(row['subject']).strip()
    pred_name = str(row['predicate']).strip()
    obj_name = str(row['object']).strip()
    img_id = str(row['image_id']).strip()

    # Skip nếu còn bất kỳ giá trị nào rỗng sau khi strip
    if not subj_name or not pred_name or not obj_name or not img_id:
        continue

    # Tạo node
    subj_node = matcher.match("Entity", name=subj_name).first() or Node("Entity", name=subj_name)
    obj_node = matcher.match("Entity", name=obj_name).first() or Node("Entity", name=obj_name)
    img_node = matcher.match("Image", id=img_id).first() or Node("Image", id=img_id)

    # Merge node
    graph.merge(subj_node, "Entity", "name")
    graph.merge(obj_node, "Entity", "name")
    graph.merge(img_node, "Image", "id")

    # Tạo & merge quan hệ
    rel1 = Relationship(subj_node, pred_name.upper(), obj_node)
    rel2 = Relationship(img_node, "HAS_OBJECT", subj_node)
    rel3 = Relationship(img_node, "HAS_OBJECT", obj_node)

    graph.merge(rel1)
    graph.merge(rel2)
    graph.merge(rel3)